# Imports

In [1]:
import os
import cv2
import numpy as np
import os
from pqdm.threads import pqdm
from functools import partial


C:\Users\narges\anaconda3\envs\tf-gpu-elina\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import tensorflow as tf

# List physical devices
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU is available: {gpus}")
else:
    print("❌ No GPU found. Running on CPU.")


✅ GPU is available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Definitions

In [5]:


def tile_image_with_padding(input_path, output_dir):
    # Load the input image
    animal_name= input_path.split('/')[-1].split(".png")[0]
    # print(animal_name)
    img = cv2.imread(input_path)
    if img is None:
        raise FileNotFoundError(f"Image not found at path: {input_path}")
    # print(img.shape)
    # if img.shape==(1400,1400,3):
    #     # Add 3-pixel zero padding on all sides
    #     padded_img = cv2.copyMakeBorder(img, 3, 3, 3, 3, borderType=cv2.BORDER_CONSTANT, value=0)
    #
    #     # Confirm padded image shape is 1406x1406
    #     height, width = padded_img.shape[:2]
    # else:
    padded_img= img
    if img.shape !=(1406,1406,3):
        # print(input_path, " size is : ", img.shape)
        # assert height == 1400 and width == 1400, "Padded image size is not 1400x1400"
        height, width = img.shape[:2]
        target_size = 1406
    
        # Calculate padding required on each side
        pad_top = (target_size - height) // 2
        pad_bottom = target_size - height - pad_top
        pad_left = (target_size - width) // 2
        pad_right = target_size - width - pad_left

        # Apply padding
        padded_img = cv2.copyMakeBorder(img, pad_top, pad_bottom, pad_left, pad_right,
                                         borderType=cv2.BORDER_CONSTANT, value=0)
        padded_img_path = os.path.join(output_dir+'padded_images/', f"{animal_name}_padded.png")
        os.makedirs(output_dir+'padded_images/', exist_ok=True)
        cv2.imwrite(padded_img_path, padded_img)
        # Confirm padded image is 1400x1400
        assert padded_img.shape == (1406, 1406, 3), \
            f"Padded image size is incorrect: {padded_img.shape}"


    # stride = 204
    # tile_size = 256
    # positions = [0, 230, 460, 690, 920, 1150]
    # positions2= [26, 256 , 486 , 716 , 946 , 1176 , 1380]
    # Draw vertical lines
    # for x in positions:
    #     cv2.line(padded_img, (x, 0), (x, padded_img.shape[0]), (0, 255, 0), 1)  # Green cut lines
    #     cv2.line(padded_img, (x + tile_size, 0), (x + tile_size, padded_img.shape[0]), (0, 0, 255), 1)  # Red overlap line
    #
    # for x in positions2:
    #     cv2.line(padded_img, (x, 0), (x, padded_img.shape[0]), (0, 255, 0), 1)  # Green cut lines
        # cv2.line(padded_img, (x + tile_size, 0), (x + tile_size, padded_img.shape[0]), (0, 0, 255), 1)  # Red overlap line

    
    # # Draw horizontal lines
    # for y in positions:
    #     cv2.line(padded_img, (0, y), (padded_img.shape[1], y), (0, 255, 0), 1)
    #     cv2.line(padded_img, (0, y + tile_size), (padded_img.shape[1], y + tile_size), (0, 0, 255), 1)


    
    # Define tiling parameters
    tile_size = 256
    stride = 204
    x_starts = [0, 230, 460, 690, 920, 1150]
    y_starts = x_starts  # since it's square

    #
    # tile_size = 512
    # # stride = 408
    # #43 overlaps
    # x_starts = [0, 444, 888]
    # y_starts = x_starts  # since it's square



    # Make sure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Generate tiles and save them
    tile_count = 0
    for y in y_starts:
        for x in x_starts:
            tile = padded_img[y:y + tile_size, x:x + tile_size]
            tile_filename = f"{animal_name}_{tile_count:03d}_x{x}_y{y}.png"
            cv2.imwrite(os.path.join(output_dir, tile_filename), tile)
            # print(output_dir+tile_filename)
            tile_count += 1

    # print(f"Saved {tile_count} tiles to '{output_dir}'")


## Example usage

In [17]:
# image_path='./Datasets/test/regions/10ld2.png'
# save_dir='./Datasets/test/regions_tiles1/'
# tile_image_with_padding(image_path, save_dir)

#
# base_address='./Datasets/'
# orig_files=['train/regions/', 'test/regions/',]
# orig_tiles_files=['train/regions/', 'test/regions/'  ]
# for i in range(len(orig_files)):
#     for filename in os.listdir(base_address+orig_files[i]):
#         tile_image_with_padding(base_address+orig_files[i]+filename, base_address+orig_tiles_files[i])



# Run the code for tilling

In [6]:
# Base path
base_address = './Dataset'

# Subdirectories
orig_files = ['/train/dic/', '/test/dic/', '/eval/dic/' ,
              '/train/orig/', '/test/orig/', '/eval/orig/' ,
              '/train/gt/', '/test/gt/',
              '/train/regions/', '/test/regions/' ]

orig_tiles_files = [ '/train/dic_tiles_256/', '/test/dic_tiles_256/', '/eval/dic_tiles_256/',
                     '/train/orig_tiles_256/', '/test/orig_tiles_256/', '/eval/orig_tiles_256/' ,
                     '/train/gt_tiles_256/', '/test/gt_tiles_256/' ,
                     '/train/regions_tiles_256/', '/test/regions_tiles_256/']

# Prepare list of (input_path, output_path) pairs
jobs = []
for i in range(len(orig_files)):
    orig_dir = base_address + orig_files[i]
    tile_dir = base_address + orig_tiles_files[i]
    for filename in os.listdir(orig_dir):
        input_path = os.path.join(orig_dir, filename)
        jobs.append((input_path, tile_dir))

def tile_wrapper(args):
    input_path, output_dir = args
    try:
        tile_image_with_padding(input_path, output_dir)
    except Exception as e:
        print(f"[ERROR] Failed on {input_path}: {e}")
        
# Run with progress bar and multiprocessing
pqdm(jobs, tile_wrapper, n_jobs=4)


QUEUEING TASKS | : 100%|██████████| 2709/2709 [00:00<00:00, 181551.00it/s]
PROCESSING TASKS | :  22%|██▏       | 604/2709 [00:16<00:56, 37.43it/s]


KeyboardInterrupt: 

# Draw tile grid lines

In [7]:
import cv2
import numpy as np

def draw_tile_grid(input_path, output_path):
    # Load the image
    img = cv2.imread(input_path)
    if img is None:
        raise FileNotFoundError(f"Image not found at path: {input_path}")
    
    # Pad with 3 pixels on all sides (zero padding)
    padded_img = cv2.copyMakeBorder(img, 3, 3, 3, 3, cv2.BORDER_CONSTANT, value=0)

    # Convert to color if grayscale, to draw colored lines
    if len(padded_img.shape) == 2 or padded_img.shape[2] == 1:
        padded_img = cv2.cvtColor(padded_img, cv2.COLOR_GRAY2BGR)

    # Parameters
    stride = 204
    tile_size = 256
    positions = [0, 230, 460, 690, 920, 1150]
    positions2= [26, 256 , 486 , 716 , 946 , 1176 , 1380]

    # tile_size = 512
    # positions = [0, 444, 888]
    # positions2= [512, 976 , 1400 ]
    # Draw vertical lines
    for x in positions:
        cv2.line(padded_img, (x, 0), (x, padded_img.shape[0]), (0, 255, 0), 1)  # Green cut lines
        cv2.line(padded_img, (x + tile_size, 0), (x + tile_size, padded_img.shape[0]), (0, 0, 255), 1)  # Red overlap line

    # for x in positions2:
    #     cv2.line(padded_img, (x, 0), (x, padded_img.shape[0]), (0, 255, 0), 1)  # Green cut lines
    #     # cv2.line(padded_img, (x + tile_size, 0), (x + tile_size, padded_img.shape[0]), (0, 0, 255), 1)  # Red overlap line

    
    # Draw horizontal lines
    for y in positions:
        cv2.line(padded_img, (0, y), (padded_img.shape[1], y), (0, 255, 0), 1)
        cv2.line(padded_img, (0, y + tile_size), (padded_img.shape[1], y + tile_size), (0, 0, 255), 1)
    for x in positions:
        for y in positions:
            cv2.putText(padded_img, "x: "+str(x) , (x + 5, y + 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 200, 255), 3, cv2.LINE_AA)
            cv2.putText(padded_img, "y: " +str(y), (x + 5, y + 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 200, 255), 3, cv2.LINE_AA)
    # Save or display
    if output_path:
        cv2.imwrite(output_path, padded_img)
        print(f"Saved image with grid to {output_path}")
    # else:
        # cv2.imshow("Tiling Grid", padded_img)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()


## Example usage

In [12]:

image_path="./Dataset/train/orig/1ld1.png"
save_dir='./Dataset/tile_test/'
draw_tile_grid(image_path, save_dir+"orig_drw_lines.png")


Saved image with grid to ./Dataset/tile_test/_dic_drw_lines.png
